In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.7 MB/s eta 0:00:00 0:00:01


In [2]:
from sentence_transformers import SentenceTransformer,util
import pandas as pd
import spacy
# from multiprocessing import Pool
import os

In [3]:
%env "TOKENIZERS_PARALLELISM" =true

env: "TOKENIZERS_PARALLELISM"=true


In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
para_path = "/kaggle/input/pclub-nlp/paragraphs.xlsx"
queries_path = "/kaggle/input/pclub-nlp/queries.xlsx"

In [6]:
paragraphs = pd.read_excel(para_path,header=None)
paragraphs = paragraphs.sample(frac=0.05)
paragraphs.head()

,0,1,2
7956,76783.0,Textual_criticism,The critic Joseph Bédier (1864–1938) launched ...
69893,123976.0,Armenians,Carpet-weaving is historically a major traditi...
58,25979.0,Zhejiang,Despite the continuing prominence of Nanjing (...
18058,120493.0,Red,Red is used in modern fashion much as it was u...
43067,126029.0,Humanism,6th-century BCE pre-Socratic Greek philosopher...


In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
sentences = []
for i in range(len(paragraphs)):
    sentences.extend(nlp(str(paragraphs.iloc[i,2])).sents)
len(paragraphs)

3753

In [9]:
# sentences = []
# def get_sentence(i):
#     return nlp(paragraphs.iloc[i,2])
# p = Pool(processes=8)
# sentences = p.map(get_sentence,range(len(paragraphs)))
# len(sentences)

In [10]:
sentences[:5]

[The critic Joseph Bédier (1864–1938) launched a particularly withering attack on stemmatics in 1928.,
 He surveyed editions of medieval French texts that were produced with the stemmatic method, and found that textual critics tended overwhelmingly to produce trees divided into just two branches.,
 He concluded that this outcome was unlikely to have occurred by chance, and that therefore, the method was tending to produce bipartite stemmas regardless of the actual history of the witnesses.,
 He suspected that editors tended to favor trees with two branches, as this would maximize the opportunities for editorial judgment (as there would be no third branch to "break the tie" whenever the witnesses disagreed).,
 He also noted that, for many works, more than one reasonable stemma could be postulated, suggesting that the method was not as rigorous or as scientific as its proponents had claimed.]

In [11]:
sentences = [str(s) for s in sentences]

In [12]:
embedings = model.encode(sentences,convert_to_tensor=True)

Batches:   0%|          | 0/587 [00:00<?, ?it/s]

In [13]:
def find_nearest_sentence(embed):
    maximum = 0
    sentence = ""
    for s,e in zip(sentences,embedings):
        sim = util.cos_sim(e,embed)
        if sim>maximum:
            maximum = sim
            sentence = s
    return maximum,sentence

In [23]:
threshold = 0.5
def get_answer(query):
    sim,ans = find_nearest_sentence(model.encode(query,convert_to_tensor=True))
    if sim>threshold:
        print(sim,ans)
    else:
        print(False)
print(get_answer("When were places named Tito?"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.6685]], device='cuda:0') Every federal unit had a town or city with historic significance from the World War II period renamed to have Tito's name included.
None


In [15]:
queries = pd.read_excel(queries_path)
queries.sample(frac=0.0001)

,What is the parts of the trophy?,True,"['- the cup itself, plus a lid and a base']"
31141,Which french emperor gained possession of Long...,True,['Napoleon III']
47028,Which which university is Fred Donner affiliated?,True,['University of Chicago']
12060,What Roman Emperor became the ruler of all of ...,True,['Augustus']
44463,What was the name of the series of agreements ...,True,['the Treaties of Tilsit']
65055,What did Pope John XXIII award Lyndon B Johnson?,False,[]
27323,Why would a knuckle -space count be used to de...,True,['language-independent']
10402,What classification did the World Cities Study...,True,['Alpha−World City']
64193,What was equal to 8.93 billion CNY?,False,[]


In [26]:
print(get_answer("What happened postwar?"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.5372]], device='cuda:0') By 1989 Germany was nearing reunification and the Cold War was coming to a close.
None
